# Imports 

In [1]:
%%capture
!pip install pandas-profiling[notebook]

In [25]:
import pandas as pd
from pandas_profiling import ProfileReport

import re
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
from sklearn.model_selection import train_test_split

import sagemaker

# Dataset 

In [63]:
# load the csv files
train = pd.read_csv("train.csv") #, header=None, names=["quadrant", "lyrics"], skiprows=1)
valid = pd.read_csv("validation.csv") #, header=None, names=["quadrant", "lyrics"], skiprows=1)
test = pd.read_csv("test.csv") #, header=None, names=["quadrant", "lyrics"], skiprows=1)

# concatenate the data
data = pd.concat([train, valid, test]).reset_index(drop=True)
# data.drop("df_index", axis=1, inplace=True)

In [64]:
data

,dzr_sng_id,valence,arousal,artist_name,track_name,quadrant,lyrics
0,213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,"Gently, hold our heads Gently, hold our heads ..."
1,216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,We are the Sun We are the dead stars We are th...
2,254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,"You're out of touch, I'm out of time But I'm o..."
3,266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,Finally close the door You'd left open wide Lo...
4,283306,0.373325,-0.923151,Duke Ellington,Caravan,3,Night and stars above that shine so bright The...
...,...,...,...,...,...,...,...
17362,113398772,-1.700572,1.842444,Sworn Enemy,We Hate,1,What I really hate is music like yours Makes m...
17363,114388304,1.178325,1.183311,Van Morrison,Wild Night,0,"As you brush your shoes, stand before the mirr..."
17364,114388306,0.373325,-0.923151,Van Morrison,Tupelo Honey,3,You can take all the tea in China Put it in a ...
17365,114388348,0.815393,0.662457,Van Morrison,Once In A Blue Moon,0,Once in a blue moon Something good comes along...


In [13]:
# perfomr a profiler/EDA report on the data
# using valid data since whole data could not fit in memory for profiling
profile = ProfileReport(valid, title="Qauadrant Lyrics Dataset", html={"style": {"full_width": True}}) #, minimal=True)
profile.to_file("EDA.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [66]:
# preprocess the lyrics column
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_preprocessing(text):
    text = text.lower() # lower text
    text = replace_contractions(text) # remove contactions
    text = "".join("".join(text).replace("\n", " ").replace("\r", " ")) # remove \n and \r
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace symbols with space
    text = BAD_SYMBOLS_RE.sub('', text) # replace bad characters with nothing
    text = re.sub(r'[0-9]', '', text) # remove residual numbers
    text = text.strip()
    text = " ".join([word for word in text.split() if word not in STOPWORDS]) # remove stopwords
    
    return text

In [67]:
data["lyrics"] = data["lyrics"].apply(text_preprocessing)

In [68]:
# get lens and remove anything with less than 64
data["len"] = data["lyrics"].apply(len)
data.describe()

,dzr_sng_id,valence,arousal,quadrant,len
count,1.736700e+04,17367.000000,17367.000000,17367.000000,17367.000000
mean,1.982202e+07,-0.069096,0.198605,1.346980,1226.929752
std,3.055292e+07,1.057003,0.959009,1.117818,11881.868344
min,2.137540e+05,-2.148097,-2.333604,0.000000,0.000000
25%,1.183880e+06,-1.040199,-0.580908,0.000000,453.000000
50%,3.783645e+06,0.032224,0.041735,1.000000,621.000000
75%,1.636354e+07,0.815393,0.846830,2.000000,846.500000
max,1.242205e+08,1.546714,2.755091,3.000000,484895.000000


In [69]:
data = data.loc[data["len"] >= 64]
data.describe()

,dzr_sng_id,valence,arousal,quadrant,len
count,1.735700e+04,17357.000000,17357.000000,17357.000000,17357.000000
mean,1.981127e+07,-0.069137,0.198658,1.346719,1227.623552
std,3.054602e+07,1.056965,0.958927,1.117736,11885.255659
min,2.137540e+05,-2.148097,-2.333604,0.000000,69.000000
25%,1.184201e+06,-1.040199,-0.579756,0.000000,453.000000
50%,3.783640e+06,0.032224,0.041735,1.000000,621.000000
75%,1.634820e+07,0.815393,0.846830,2.000000,847.000000
max,1.242205e+08,1.546714,2.755091,3.000000,484895.000000


In [70]:
data.drop(labels=["len"], axis=1, inplace=True)

C:\Users\marti\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [71]:
# split the preprocessed data
train_test = 0.1
train_valid = 0.8

train, test = train_test_split(data, test_size=train_test, random_state=0)
train, valid = train_test_split(train, train_size=train_valid, random_state=0)

print(train.shape, test.shape, valid.shape)

(12496, 7) (1736, 7) (3125, 7)


In [72]:
train.to_csv("preprocessed_train.csv", index=False)
test.to_csv("preprocessed_test.csv", index=False)
valid.to_csv("preprocessed_valid.csv", index=False)

In [73]:
# generate lyrics.txt and labels.txt for all datasets
def aggregate_lyrics(dataset, lyrics_path, labels_path):
    """
    Aggregate lyrics and their respective labels / quadrant
    :param dataset: path to data
    :param lyrics_path: path to lyrics .txt file
    :param labels_path: path to labels .txt file
    :return: None
    """
    
    data = pd.read_csv(dataset)
    failed_index = []   # indexes with faulty data
    
    lyrics = data["lyrics"]
    labels = data["quadrant"]
    
    with open(lyrics_path, "w") as f:
        for _ in range(lyrics.shape[0]):
            try:
                f.write("".join("".join(lyrics[_]).replace("\n", " ").replace("\r", " "))+"\n")
            except UnicodeEncodeError:
                failed_index.append(_)
                
    with open(labels_path, "w") as f:
        for _ in range(labels.shape[0]):
            try:
                if _ not in failed_index:
                    f.write(str(labels.iloc[_])+"\n")
            except UnicodeEncodeError:
                pass

In [74]:
dataset_list = ["preprocessed_train.csv", "preprocessed_test.csv", "preprocessed_valid.csv"]
lyrics_path_list = ["data/train/lyrics.txt", "data/test/lyrics.txt", "data/valid/lyrics.txt"]
labels_path_list = ["data/train/labels.txt", "data/test/labels.txt", "data/valid/labels.txt"]

In [75]:
for i in range(len(dataset_list)):
    aggregate_lyrics(dataset_list[i], lyrics_path_list[i], labels_path_list[i])

In [ ]:
# upload data
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()

print("Default Bucket: {}".format(bucket))
print("RoleArn: {}".format(role))

In [ ]:
local_dir = "data"
prefix = "emotion_recognition_music_lyrics"
inputs = session.upload_data(path=local_dir, bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

In [ ]:
# Saving this to avoid uploading the inputs again)
inputs = "s3://sagemaker-us-east-1-420657680415/emotion_recognition_music_lyrics"
print("input spec (in this case, just an S3 path): {}".format(inputs))